# Transfer Learning for Computer Vision Tutorial

Based on the tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

This notebook provides an introduction to transfer learning for computer vision using PyTorch. It demonstrates how to leverage pre-trained models to classify images of ants and bees from the Hymenoptera dataset.

The dataset is loaded using a custom [HymenopteraData](../datasets/hymenoptera_data.py) class, which organizes the data into training and validation sets. These sets are then loaded using `DataLoader` with a specified batch size.

A pre-trained ResNet18 model is loaded using `torchvision.models`, and the final fully connected layer is replaced to match the number of classes in the Hymenoptera dataset. The model is trained using the SGD optimizer with momentum and a cross-entropy loss function. A learning rate scheduler (`StepLR`) is used to adjust the learning rate at regular intervals.

The training loop iterates over the dataset for a specified number of epochs, updating the model parameters to minimize the loss. After each epoch, the model is evaluated on the validation set to measure its accuracy. The evaluation loop computes the accuracy by comparing the model's predictions with the true labels.

Additionally, the notebook includes code to save the trained model's state dictionary to a file and load it back for inference. This allows for the model to be reused without retraining.

The notebook concludes with a section on visualizing the model's prediction on a validation image, providing a qualitative assessment of the model's performance. In fact, the image is not an ant or a bee, it's a butterfly. The model was trained to classify ants or bees, so it classified the butterfly as a bee, probably because it's more similar (it has wings, for example). According to the path of the image (it's in the `bees` folder), the model classified correctly (among the options). This is the same validation image used in the original tutorial.

Most of the boilerplate for the training is handled by the manager class. This includes printing metrics during training and periodically saving the model, allowing training to be resumed from the last checkpoint.

The executor `GeneralBatchExecutor` is the default and simplest executor that expects a tensor as input, runs the model with that input, and returns the output. A custom evaluator, `MyEvaluator`, was defined to get the path of an image as input, retrieve the image, transform it into a tensor, run the model with this input tensor, and obtain the insect class from the output. The evaluator then displays the image along with the predicted class.

A custom metrics class, `MyMetrics`, was defined to display 6 predictions on images from the validation dataset after the model was trained. A metrics class can define data to be persisted along with the model weights as the result of the `run` method, in a dictionary containing only primitive data types, dictionaries, and lists that can be serialized and parsed directly. This specific class (`MyMetrics`) does not need to store any data and was used solely to plot the images, which are included in the PDF saved at `REPORT_PATH`.

More metrics classes can be seen at [lib/metrics.py](../lib/metrics.py).

In [ ]:
NAME = 'transfer_learning'
SAVE_PATH = f'data/test/train/{NAME}-checkpoint.pth'
REPORT_PATH = f'data/test/train/{NAME}-report.pdf'
DATA_PATH = 'data/hymenoptera_data'

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [10]:
import sys
if "../" not in sys.path:
  sys.path.append("../")

In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import models
import matplotlib.pyplot as plt
from PIL import Image

cudnn.benchmark = True
plt.ion()   # interactive mode

### Load Data

In [ ]:
import typing
from src.datasets.hymenoptera_data import HymenopteraData

data = HymenopteraData(root_path=DATA_PATH)
datasets = data.datasets

print(
    "Loading HymenopteraData",
    len(datasets.train) if isinstance(datasets.train, typing.Sized) else None,
    len(datasets.validation or []) if isinstance(datasets.validation, typing.Sized) else None,
    len(datasets.test or []) if isinstance(datasets.test, typing.Sized) else None)

In [13]:
image_datasets = dict(
    train=datasets.train,
    val=datasets.validation,
)
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x],
        batch_size=4,
        shuffle=True,
        num_workers=4,
    ) for x in ['train', 'val']
}
dataset_sizes = data.dataset_sizes
class_names = data.class_names

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Visualize a few images

In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

### Finetuning the ConvNet

In [15]:
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Visualizing the model predictions

In [16]:
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from auto_mind.supervised.data import MinimalFullState
from src.lib.metrics import MetricsItemPlotter, MetricsCalculatorParams

class MyMetrics(MetricsItemPlotter[dict[str, None]]):
    def __init__(self, name: str, num_images=6):
        super().__init__(name=name)
        self.num_images = num_images
        self.model: torch.nn.Module | None = None

    def run(self, params: MetricsCalculatorParams) -> dict[str, None]:
        self.model = params.model
        return dict()

    def plot(self, info: MinimalFullState, metric: dict[str, None], figsize: tuple[float, float] | None) -> list[Figure]:
        model = self.model
        assert model is not None
        figs = self.visualize_model(model, figsize=figsize)
        plt.ioff()
        plt.show()
        return figs

    def visualize_model(self, model: torch.nn.Module, figsize: tuple[float, float] | None) -> list[Figure]:
        num_images = self.num_images
        was_training = model.training
        model.eval()
        images_so_far = 0
        fig = plt.figure(figsize=figsize)
        figs = [fig]

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(dataloaders['val']):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                for j in range(inputs.size()[0]):
                    images_so_far += 1
                    ax = plt.subplot(num_images//2, 2, images_so_far)
                    ax.axis('off')
                    ax.set_title(f'predicted: {class_names[preds[j]]}')
                    imshow(inputs.cpu().data[j])

                    if images_so_far == num_images:
                        model.train(mode=was_training)
                        return figs
                    else:
                        img = plt.gcf()
                        figs.append(img)
            model.train(mode=was_training)

        return figs

### Create the Manager

In [17]:
from auto_mind import supervised
from auto_mind.supervised.handlers import GeneralBatchExecutor, GeneralBatchAccuracyCalculator, Evaluator
from src.lib.metrics import MetricsListPlotter, MainMetrics, MetricsFileDirectPlotter

class MyEvaluator(Evaluator[str, None]):
    def run(self, params) -> None:
        model = params.model
        img_path = params.input
        self.visualize_model_predictions(model, img_path)

    def visualize_model_predictions(self, model: torch.nn.Module, img_path: str):
        img = Image.open(img_path)
        img: typing.Any = data.data_transforms['val'](img)
        input: torch.Tensor = img
        input = input.unsqueeze(0)
        input = input.to(device)

        outputs = model(input)
        _, preds = torch.max(outputs, 1)

        ax = plt.subplot(2,2,1)
        ax.axis('off')
        ax.set_title(f'Predicted: {class_names[preds[0]]}')
        imshow(input.cpu().data[0])

        plt.ioff()
        plt.show()

manager = supervised.Manager(
    data_params=supervised.ManagerDataParams(
        train_dataloader=dataloaders['train'],
        validation_dataloader=dataloaders['val'],
        test_dataloader=None,
    ),
    model_params=supervised.ManagerModelParams(
        model=model_ft,
        criterion=criterion,
        executor=GeneralBatchExecutor(),
        use_best=True,
    ),
    optimizer_params=supervised.ManagerOptimizerParams(
        optimizer=optimizer_ft,
        scheduler=exp_lr_scheduler,
        train_early_stopper=None,
        test_early_stopper=None,
    ),
    metrics_params=supervised.ManagerMetricsParams(
        evaluator=MyEvaluator(),
        accuracy_calculator=GeneralBatchAccuracyCalculator(),
        metrics_calculator=MetricsFileDirectPlotter(
            plotter=MetricsListPlotter(items=[
                MainMetrics(name=NAME),
                MyMetrics(name=f"{NAME}_view", num_images=6),
            ]),
            file_path=REPORT_PATH,
        ),
        batch_interval=False,
        default_interval=1,
    ),
    config=supervised.ManagerConfig(
        save_path=SAVE_PATH,
        random_seed=0,
        train_hook=None,
    ),
)

### Train and evaluate

In [ ]:
manager.train(epochs=25)
info = manager.info()
assert info is not None
print(f'Best val Acc: {info.train_results.best_accuracy:4f}')

### Inference on custom images

In [ ]:
manager.evaluate(f'{data.data_dir}/val/bees/72100438_73de9f17af.jpg')